# Preparation

I use Python 3, but everything should work with Python 2.

1. Install [HDF5](https://www.hdfgroup.org/HDF5/release/obtain5.html).
2. Install other packages:

<code>pip install h5py keras matplotlib numpy pyyaml scipy scitkit-learn theano urllib3</code>

# Tutorial

The goal of this project is to learn distributed representations of MLB players, which can then be used for other types of analyses. The project is inspired by [word2vec](https://en.wikipedia.org/wiki/Word2vec) (hence the name), which learns distributed representations of words. These distributed representations often have pretty interesting properties; for example, Paris - France + Italy = Rome (see [here](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) and [here](http://arxiv.org/pdf/1301.3781.pdf) for more details).

In this notebook, I'll show you how I built a model that simultaneously learns distributed representations of pitchers and batters from at bat data.

To start things off, let's download some data from [Retrosheet.org](http://retrosheet.org/). We'll use data from the noughties.

In [ ]:
import urllib.request

urllib.request.urlretrieve("http://www.retrosheet.org/events/2000seve.zip", "2000seve.zip")

Now, extract the data.

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("2000seve.zip", "r")
zip_ref.extractall("2000seve")
zip_ref.close()

Next, we'll prepare some variables for collecting the data.

In [1]:
from os import listdir
from os.path import isfile, join

data_directory = "/home/airalcorn2/Projects/Deep Baseball/2000seve"
data_files = [f for f in listdir(data_directory) if isfile(join(data_directory, f))]
data = []
at_bats = {}
home_runs = {}
singles = {}
doubles = {}
counts = {"batter": {}, "pitcher": {}}

And now we'll read in the data. Unfortunately, this is going to be a bunch of spaghetti code. The goal is to collect the batter, pitcher, and pitch outcome (e.g., strike, ball, double) for every pitch. By the end of the following code block, we'll have a Python list of dictionaries with the format <code>{"batter": batter, "pitcher": pitcher, "outcome": outcome}</code>. To best understand what's going on in the code, you'll have to read through Retrosheet's [game file documentation](http://www.retrosheet.org/game.htm).

In [2]:
import string

for data_file in data_files:
    # Skip non-event files.
    if not (".EVA" in data_file or ".EVN" in data_file):
        continue
    
    f = open(join(data_directory, data_file))
    home_pitcher = None
    away_pitcher = None
    line = f.readline().strip()
    
    while line != "":
        parts = line.split(",")
        
        # Get starting pitchers.
        if parts[0] == "id":
            while parts[0] != "play":
                line = f.readline().strip()
                parts = line.split(",")
                if parts[0] == "start" and parts[-1] == "1":
                    if parts[3] == "0":
                        away_pitcher = parts[1]
                    else:
                        home_pitcher = parts[1]
        
        # Skip non-plays, steals, errors on foul fly balls, 
        # picked off stealings, and other random plays.
        if (parts[-1] == "NP" or parts[-1][:2] == "CS" or parts[-1][:2] == "DI" or
            parts[-1][:2] == "SB" or parts[-1][:3] == "FLE" or parts[-1][:4] == "POCS" or
            parts[-1][:2] == "OA"):
            line = f.readline().strip()
            continue
        
        # Get at bat data.
        if parts[0] == "play":
            batter = parts[3]
            pitcher = home_pitcher
            if parts[2] == "1":
                pitcher = away_pitcher
            
            at_bats[batter] = at_bats.get(batter, 0) + 1
            counts["batter"][batter] = counts["batter"].get(batter, 0) + 1
            counts["pitcher"][pitcher] = counts["pitcher"].get(pitcher, 0) + 1
            
            row = {"batter": batter, "pitcher": pitcher}
            
            # Handle balks, wild pitches, passed balls, and pickoffs.
            if (parts[-1][:2] == "BK" or parts[-1][:2] == "WP" or
                parts[-1][:2] == "PB" or parts[-1][:2] == "PO"):
                row["outcome"] = parts[-1][:2]
                data.append(row)
                line = f.readline().strip()
                continue
            
            # Cycle through the piches for the current at bat.
            # See "The pitches field of the play record" here: http://www.retrosheet.org/eventfile.htm.
            pitches = parts[5]
            i = 0
            while i < len(pitches):
                pitch = pitches[i]
                
                # Handle catcher pickoffs, pitches blocked by catcher, or runners.
                if pitch == "+" or pitch == ">":
                    i += 1
                    pitch = pitches[i]
                elif pitch == "*":
                    i += 1
                    pitch += pitches[i]
                
                if "X" not in pitch and pitch != "." and pitch != "*" and pitch != "+":
                    row["outcome"] = pitch
                    data.append(row)
                
                i += 1
            
            # If the last pitch resulted in contact, figure out the pitch outcome.
            # See "Events made by the batter at the plate" here: http://www.retrosheet.org/eventfile.htm#8.
            if pitches[-1] == "X":
                play_parts = parts[6].split("/")
                main_play = play_parts[0]
                play = main_play.split(".")[0]
                
                if play[0] == "H":
                    play = "HR"
                elif play[0] in string.digits:
                    play = play[0]
                elif play[0] in {"S", "D", "T"}:
                    play = play[:2]
                    # Try to get first ball handler.
                    if len(play) < 2:
                        try:
                            handlers = play_parts[1]
                            play = play[0] + handlers[0]
                        except IndexError:
                            pass
                
                row["outcome"] = play
                if play == "HR":
                    home_runs[batter] = home_runs.get(batter, 0) + 1
                elif play[0] == "S":
                    singles[batter] = singles.get(batter, 0) + 1
                elif play[0] == "D":
                    doubles[batter] = doubles.get(batter, 0) + 1
                
                data.append(row)
        
        # Handle pitching changes.
        if parts[0] == "sub":
            if parts[-1] == "1":
                if parts[3] == "0":
                    away_pitcher = parts[1]
                else:
                    home_pitcher = parts[1]
        
        line = f.readline().strip()

OK, now that we have our raw data, we're going to establish some cutoffs so that we're only analyzing players with a reasonable amount of data. We're going to only include the most frequent batters and pitchers (i.e., those who accounted for 90% of the pitches).

In [3]:
cutoffs = {}
percentile_cutoff = 0.9
for player_type in ["batter", "pitcher"]:
    counts_list = list(counts[player_type].values())
    counts_list.sort(reverse = True)
    total_pitches = sum(counts_list)
    cumulative_percentage = [sum(counts_list[:i + 1]) / total_pitches for i in range(len(counts_list))]
    cutoff_index = sum([1 for total in cumulative_percentage if total <= percentile_cutoff])
    cutoff = counts_list[cutoff_index]
    cutoffs[player_type] = cutoff
    print(player_type)
    print("Original: {0}\tNew: {1}\tProportion: {2:.2f}".format(
            len(counts[player_type]), cutoff_index, cutoff_index / len(counts[player_type])))

batter
Original: 2699	New: 715	Proportion: 0.26
pitcher
Original: 1760	New: 774	Proportion: 0.44


As you can see, only 26% of batters and 44% of pitchers were involved in 90% of the pitches.

Let's use these new cutoff points to build the final data set.

In [4]:
final_data = []
for sample in data:
    batter = sample["batter"]
    pitcher = sample["pitcher"]
    if counts["batter"][batter] >= cutoffs["batter"] and counts["pitcher"][pitcher] >= cutoffs["pitcher"]:
        final_data.append(sample)

print("Original: {0}\tReduced: {1}".format(len(data), len(final_data)))

Original: 7204897	Reduced: 5867152


As you can see, we still have a large data set even after removing rare batters and pitchers.

Next, we're going to associate an integer index with each of our batters, pitchers, and outcomes, respectively.

In [5]:
import random

random.shuffle(final_data)

categories = {"batter": set(), "pitcher": set(), "outcome": set()}
for sample in final_data:
    categories["batter"].add(sample["batter"])
    categories["pitcher"].add(sample["pitcher"])
    categories["outcome"].add(sample["outcome"])

for column in categories:
    categories[column] = list(categories[column])
    categories[column].sort()

category_to_int = {}
for column in categories:
    category_to_int[column] = {categories[column][i]: i for i in range(len(categories[column]))}

We'll then use these newly defined integer indices to build the appropriate NumPy arrays for our model.

In [6]:
import numpy as np
from keras.utils import np_utils

BATCH_SIZE = 100
NUM_BATTERS = len(categories["batter"])
NUM_PITCHERS = len(categories["pitcher"])
NUM_OUTCOMES = len(categories["outcome"])
VEC_SIZE = 50

data_sets = {"batter": [], "pitcher": [], "outcome": []}
for sample in final_data:
    for column in sample:
        value = sample[column]
        value_index = category_to_int[column][value]
        data_sets[column].append(value_index)

for column in data_sets:
    data_sets[column] = np.array(data_sets[column])

data_sets["outcome"] = np_utils.to_categorical(data_sets["outcome"], NUM_OUTCOMES)

Using Theano backend.


We're now ready to build our model with [Keras](http://keras.io/) and [Theano](http://deeplearning.net/software/theano/). The model is similar in spirit to a word2vec model in that we're trying to learn the player embeddings that best predict the outcome of a pitch (the "target word" in word2vec) given a certain batter and pitcher (the "context" in word2vec). We'll learn separate embedding matrices for batters and pitchers.

In [9]:
from keras.layers import Embedding, Dropout, Merge
from keras.layers.core import Dense, Reshape
from keras.models import Sequential

batter_embed = Sequential()
batter_embed.add(Embedding(NUM_BATTERS, VEC_SIZE, input_length = 1))
batter_embed.add(Reshape((VEC_SIZE,)))

pitcher_embed = Sequential()
pitcher_embed.add(Embedding(NUM_PITCHERS, VEC_SIZE, input_length = 1))
pitcher_embed.add(Reshape((VEC_SIZE,)))

model = Sequential()
model.add(Merge([batter_embed, pitcher_embed], mode = "concat"))
model.add(Dense(NUM_OUTCOMES, activation = "softmax"))
# model.add(Dropout(0.5))
model.compile(optimizer = "sgd", loss = "categorical_crossentropy",
              metrics = ["accuracy"])

And we're now ready to train our model. We'll save the weights at the end of training.

In [10]:
X_list = [data_sets["batter"].reshape(data_sets["batter"].shape[0], 1),
          data_sets["pitcher"].reshape(data_sets["pitcher"].shape[0], 1)]
y = data_sets["outcome"]
model.fit(X_list, y, nb_epoch = 100, batch_size = 100, verbose = 2, shuffle = True)
model.save_weights("weights.hdf5")
model.load_weights("weights.hdf5")

Epoch 1/100
88s - loss: 3.0684 - acc: 0.1517
Epoch 2/100
95s - loss: 2.9515 - acc: 0.1572
Epoch 3/100
93s - loss: 2.9258 - acc: 0.1638
Epoch 4/100
99s - loss: 2.9133 - acc: 0.1662
Epoch 5/100
97s - loss: 2.9060 - acc: 0.1671
Epoch 6/100
104s - loss: 2.9010 - acc: 0.1675
Epoch 7/100
103s - loss: 2.8972 - acc: 0.1678
Epoch 8/100
98s - loss: 2.8942 - acc: 0.1679
Epoch 9/100
104s - loss: 2.8919 - acc: 0.1680
Epoch 10/100
110s - loss: 2.8899 - acc: 0.1681
Epoch 11/100
101s - loss: 2.8882 - acc: 0.1683
Epoch 12/100
100s - loss: 2.8867 - acc: 0.1682
Epoch 13/100
99s - loss: 2.8853 - acc: 0.1683
Epoch 14/100
107s - loss: 2.8841 - acc: 0.1682
Epoch 15/100
100s - loss: 2.8829 - acc: 0.1683
Epoch 16/100
102s - loss: 2.8817 - acc: 0.1683
Epoch 17/100
100s - loss: 2.8807 - acc: 0.1683
Epoch 18/100
103s - loss: 2.8797 - acc: 0.1684
Epoch 19/100
107s - loss: 2.8788 - acc: 0.1684
Epoch 20/100
100s - loss: 2.8779 - acc: 0.1684
Epoch 21/100
103s - loss: 2.8771 - acc: 0.1684
Epoch 22/100
104s - loss: 2.8

Having trained the model, let's go ahead and get the distributed representations for all of our players. In order to do so, we need to define some functions that return an embedding when provided with a player's integer index.

In [11]:
from keras import backend

get_batter_vec = backend.function([batter_embed.input], batter_embed.output)
get_pitcher_vec = backend.function([pitcher_embed.input], pitcher_embed.output)

batter_vecs = [get_batter_vec([np.array([[i]])]) for i in range(NUM_BATTERS)]
pitcher_vecs = [get_pitcher_vec([np.array([[i]])]) for i in range(NUM_PITCHERS)]

# Get distributed representation of players.
batter_vecs = np.array(batter_vecs).reshape((NUM_BATTERS, VEC_SIZE))
pitcher_vecs = np.array(pitcher_vecs).reshape((NUM_PITCHERS, VEC_SIZE))
player_vecs = {"batter": batter_vecs, "pitcher": pitcher_vecs}

Now, let's find out if these embeddings are doing anything interesting. First, let's collect some information about the players.

In [12]:
# Get player data.
player_data = {}

for data_file in data_files:
    if ".ROS" in data_file:
        f = open(join(data_directory, data_file))
        for line in f:
            parts = line.strip().split(",")
            player_id = parts[0]
            last_name = parts[1]
            first_name = parts[2]
            name = first_name + " " + last_name
            batting_hand = parts[3]
            throwing_hand = parts[4]
            position = parts[6]
            player_data[player_id] = {"name": name, "batting_hand": batting_hand,
                                      "throwing_hand": throwing_hand, "position": position}

Here, we're going to use the [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) algorithm to visualize the embeddings.

In [13]:
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

NUM_PLAYERS = {"batter": NUM_BATTERS, "pitcher": NUM_PITCHERS}


def run_tsne(player_type):    
    player_names = []
    for i in range(NUM_PLAYERS[player_type]):
        player_id = categories[player_type][i]
        player_names.append(player_data[player_id]["name"])
    
    tsne = TSNE(n_components = 3, learning_rate = 100)
    manifold_3d = tsne.fit_transform(player_vecs[player_type])
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = "3d")
    ax.scatter(manifold_3d[:, 0], manifold_3d[:, 1], manifold_3d[:, 2])
    
    plt.show()
    
    tsne = TSNE(n_components = 2, learning_rate = 100)
    manifold_2d = tsne.fit_transform(player_vecs[player_type])
    (x, y) = (manifold_2d[:, 0], manifold_2d[:, 1])
    
    plt.scatter(x, y)
    for i in range(len(player_names)):
        plt.text(x[i], y[i], player_names[i], va = "top", family = "monospace")

    plt.show()
    
    return manifold_3d


tsne_batters = run_tsne("batter")
tsne_pitchers = run_tsne("pitcher")

Next, we're going to perform a [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) on the embeddings and color them with various interesting properties.

In [14]:
from sklearn import decomposition


def run_pca(player_vecs, colors = None, pc_x = 0, pc_y = 1, pc_z = 2, do_print = False, title = ""):
    """
    Run a PCA on the embedded player representations.
    :param player_vecs: 
    :param colors: 
    :param pc_x: 
    :param pc_y: 
    :param pc_z: 
    :return: 
    """
    pca = decomposition.PCA()
    pca.fit(player_vecs)
    if do_print:
        print(pca.explained_variance_ratio_)
    
    projected = pca.transform(player_vecs)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = "3d")
    ax.scatter(projected[:, pc_x], projected[:, pc_y], projected[:, pc_z], color = colors)
    ax.set_title(title)
    
    plt.show()
    
    plt.scatter(projected[:, pc_x], projected[:, pc_y], c = colors, cmap = "gray")
    plt.title(title)
    plt.show()
    return projected


max_hr_rate = max([home_runs.get(batter_id, 0) / at_bats[batter_id] for batter_id in at_bats if batter_id in categories["batter"]])
max_single_rate = max([singles.get(batter_id, 0) / at_bats[batter_id] for batter_id in at_bats if batter_id in categories["batter"]])
max_double_rate = max([doubles.get(batter_id, 0) / at_bats[batter_id] for batter_id in at_bats if batter_id in categories["batter"]])

batting_hand_color = {"L": "red", "R": "green", "B": "purple"}
batter_colors = {"hand": [], "hr": [], "single": [], "double": []}
for i in range(NUM_BATTERS):
    batter_id = categories["batter"][i]
    batting_hand = player_data[batter_id]["batting_hand"]
    batter_colors["hand"].append(batting_hand_color[batting_hand])
    batter_colors["hr"].append(str((home_runs.get(batter_id, 0) / at_bats[batter_id]) / max_hr_rate))
    batter_colors["single"].append(str((singles.get(batter_id, 0) / at_bats[batter_id]) / max_single_rate))
    batter_colors["double"].append(str((doubles.get(batter_id, 0) / at_bats[batter_id]) / max_double_rate))

for batter_color in ["hand", "single", "double", "hr"]:
    projected_batters = run_pca(batter_vecs, batter_colors[batter_color], title = "batter_{0}".format(batter_color))
    no = run_pca(batter_vecs, batter_colors[batter_color], 1, 2, 3)

projected_batters = run_pca(batter_vecs, batter_colors["hr"], do_print = True)
projected_pitchers = run_pca(pitcher_vecs, do_print = True)

[ 0.13223021  0.12286694  0.08537145  0.0525946   0.04789656  0.0431582
  0.03850202  0.03790293  0.03516199  0.03114783  0.02675426  0.02388979
  0.02160699  0.02118008  0.01953577  0.01837644  0.0179307   0.01772697
  0.01635154  0.01565541  0.01439157  0.01396391  0.01347487  0.01327055
  0.01259823  0.01164937  0.01021037  0.01007907  0.00949579  0.00927316
  0.00699846  0.00673589  0.00477701  0.0045842   0.00364564  0.00348533
  0.00271707  0.00254637  0.0024559   0.00217474  0.00198852  0.00177959
  0.00172125  0.00170362  0.00165129  0.00149731  0.00141244  0.00137111
  0.00129052  0.00121625]
[ 0.12636515  0.07242899  0.06970441  0.05834733  0.04754408  0.04151186
  0.03415529  0.03311685  0.03091065  0.0292076   0.02853769  0.02628876
  0.02476904  0.0240183   0.02371372  0.02331884  0.02278794  0.02082521
  0.02048968  0.01961536  0.01826623  0.01791103  0.01655461  0.01629054
  0.0149606   0.01438989  0.01328173  0.01292827  0.01232073  0.01130352
  0.01006282  0.00779779  

As you can see, there are some interesting patterns in the embeddings. For example, right-handed hitters are clearly separated from left-handed and switch hitters.

<img src="batters_hand_pca.png">

Similarly, frequent singles hitters are far from infrequent singles hitters.

<img src="batters_singles_pca.png">

So, the model is clearly learning something, but whether or not what it's learning is non-trivial remains to be seen.

Let's go ahead and save those PC scores in a CSV to play around with elsewhere if we want.

In [15]:
import csv


def write_projected_data(player_type, projected, fieldnames, projection):
    """
    Write the projected scores of the players to a file.
    :param player_type: 
    :param projected: 
    :param fieldnames: 
    :return: 
    """
    out = open("{0}s_{1}.csv".format(player_type, projection), "w")
    output = csv.DictWriter(out, fieldnames = fieldnames)
    output.writeheader()
    
    for i in range(NUM_PLAYERS[player_type]):
        player_id = categories[player_type][i]
        row = {}
        for col in fieldnames:
            if col in player_data[player_id]:
                row[col] = player_data[player_id][col]
        
        xyz = ["x", "y", "z"]
        for j in range(3):
            if projection == "pca":
                row["PC{0}".format(j + 1)] = projected[i][j]
            else:
                row[xyz[j]] = projected[i][j]
        
        row["player_id"] = player_id
        if player_type == "batter":
            row["hr_rate"] = home_runs.get(player_id, 0) / at_bats[player_id]
        
        nothing = output.writerow(row)
    
    out.close()


fieldnames = ["player_id", "name", "position", "batting_hand", "throwing_hand", "hr_rate", "PC1", "PC2", "PC3"]
write_projected_data("batter", projected_batters, fieldnames, "pca")
write_projected_data("batter", tsne_batters, fieldnames[:-3] + ["x", "y", "z"], "tsne")

fieldnames = ["player_id", "name", "throwing_hand", "PC1", "PC2", "PC3"]
write_projected_data("pitcher", projected_pitchers, fieldnames, "pca")
write_projected_data("pitcher", tsne_pitchers, fieldnames[:-3] + ["x", "y", "z"], "tsne")

To get a better sense of the embeddings, I recommend exploring the PC scores in my open source [ScatterPlot3D](https://sites.google.com/site/michaelaalcorn/ScatterPlot3D) software. To run it:

1. Download the appropriate build.
2. Run with <code>java -jar ScatterPlot3D-&lt;version&gt;.jar</code> on Linux systems or by double-clicking the JAR on Windows.
3. Load the data.
4. Put 4, 5, and 6 for x, y, and z for "pitchers_pca.csv" or 7, 8, and 9 for "batters_pca.csv".
5. Click "Submit".

You can then search, zoom, and rotate the data or click on individual points for more details. For example:

<img src="pitchers_tsne_all.png">

<img src="pedro_martinez.png">

Documentation can be downloaded [here](https://sites.google.com/site/michaelaalcorn/ScatterPlot3D/SupplementaryMaterials.zip?attredirects=0&d=1). A gallery of application screenshots can be found [here](http://imgur.com/a/U833y).

We'll also save the player embeddings.

In [16]:
def write_distributed_representations(player_type, player_vecs):
    """
    Write the hidden vector representation of the players to a file.
    :param player_type: 
    :param player_vecs: 
    :return: 
    """
    out = open("{0}s_latent.csv".format(player_type), "w")
    fieldnames = ["name"] + ["latent_{0}".format(i + 1) for i in range(VEC_SIZE)]
    output = csv.DictWriter(out, fieldnames = fieldnames)
    output.writeheader()
    
    for i in range(NUM_PLAYERS[player_type]):
        player_id = categories[player_type][i]
        row = {"name": player_data[player_id]["name"]}
        
        for j in range(VEC_SIZE):
            row["latent_{0}".format(j + 1)] = player_vecs[i][j]
        
        nothing = output.writerow(row)
    
    out.close()


write_distributed_representations("batter", batter_vecs)
write_distributed_representations("pitcher", pitcher_vecs)

So, do the embeddings contain any non-obvious information? Maybe comparing nearest neighbors will provide some insight.

In [17]:
import pandas as pd


def get_nearest_neighbors(name, data, latent_vecs, player_names, k = 10):
    """
    Print the k nearest neighbors (in the latent space) of a given player.
    :param name: 
    :param k: 
    :return: 
    """
    player_index = np.where(data["name"] == name)[0]
    player_latent = latent_vecs[player_index]
    distances = list(np.linalg.norm(latent_vecs - player_latent, axis = 1))
    distances_and_ids = list(zip(player_names, distances))
    distances_and_ids.sort(key = lambda x: x[1])
    
    return distances_and_ids[1:1 + k]


data_files = ["batters_latent.csv", "pitchers_latent.csv"]
data = {}
player_names = {}
latent_vecs = {}
for player_type in ["batter", "pitcher"]:
    data_file = "{0}s_latent.csv".format(player_type)
    data[player_type] = pd.read_csv(data_file)
    player_names[player_type] = list(data[player_type]["name"])
    latent_vecs[player_type] = np.array(data[player_type].iloc[:, 1:])

print("Barry Bonds")
print(get_nearest_neighbors("Barry Bonds", data["batter"], latent_vecs["batter"], player_names["batter"]))
print()

print("Ichiro Suzuki")
print(get_nearest_neighbors("Ichiro Suzuki", data["batter"], latent_vecs["batter"], player_names["batter"]))
print()

print("Bartolo Colon")
print(get_nearest_neighbors("Bartolo Colon", data["pitcher"], latent_vecs["pitcher"], player_names["pitcher"]))
print()

print("Barry Zito")
print(get_nearest_neighbors("Barry Zito", data["pitcher"], latent_vecs["pitcher"], player_names["pitcher"]))
print()

Barry Bonds
[('Lance Berkman', 0.80362719512503178), ('Luis Gonzalez', 0.81064423585497591), ('Rafael Palmeiro', 0.83048049922199019), ('Carlos Delgado', 0.83236090089884773), ('Erubiel Durazo', 0.85260775960412272), ('Larry Walker', 0.86558149817812369), ('Mark Teixeira', 0.86814735972554424), ('Jim Thome', 0.88391322107211046), ('Brian Giles', 0.88633155655171592), ('Jason Giambi', 0.88935383504254895)]

Ichiro Suzuki
[('Jose Reyes', 0.87172853791218863), ('Melky Cabrera', 0.8774168578851943), ('Ross Gload', 0.88014064020989569), ('Aaron Miles', 0.89513516389704362), ('Jacoby Ellsbury', 0.91225106292926716), ('James Loney', 0.91241217495568494), ('Carl Crawford', 0.91697925056492258), ('Pablo Sandoval', 0.92538678486094827), ('Cesar Izturis', 0.92626444147775433), ('Jose Vizcaino', 0.93230174803919452)]

Bartolo Colon
[('Rodrigo Lopez', 0.54489953082561748), ('Andy Sonnanstine', 0.54990961752142942), ('Ben Sheets', 0.55292689441319642), ('Elmer Dessens', 0.57579324764653239), ('Gil M

Unfortunately, my rather limited baseball knowledge means I do not know the answer to that question. Maybe you can tell me? We can also combine players.

In [18]:
def combine_players(player_1_name, player_2_name, data, latent_vecs, player_names, k = 10, subtract = False):
    """
    Print the k nearest neighbors of the vector resulting from combining two
    players in the latent space.
    :param player_1_name: 
    :param player_2_name: 
    :param k: 
    :param subtract: 
    :return: 
    """
    player_1_index = np.where(data["name"] == player_1_name)[0]
    player_1_latent = latent_vecs[player_1_index]
    
    player_2_index = np.where(data["name"] == player_2_name)[0]
    player_2_latent = latent_vecs[player_2_index]
    
    distances = list(np.linalg.norm(latent_vecs - (player_1_latent + player_2_latent), axis = 1))
    if subtract:
        distances = list(np.linalg.norm(latent_vecs - (player_1_latent - player_2_latent), axis = 1))
    
    distances_and_ids = list(zip(player_names, distances))
    distances_and_ids.sort(key = lambda x: x[1])
    return distances_and_ids[1:1 + k]


print("Barry Bonds + Ichiro Suzuki")
print(combine_players("Barry Bonds", "Ichiro Suzuki", data["batter"], latent_vecs["batter"], player_names["batter"]))
print()

print("Barry Bonds - Ichiro Suzuki")
print(combine_players("Barry Bonds", "Ichiro Suzuki", data["batter"], latent_vecs["batter"], player_names["batter"], subtract = True))
print()

print("Ichiro Suzuki - Barry Bonds")
print(combine_players("Ichiro Suzuki", "Barry Bonds", data["batter"], latent_vecs["batter"], player_names["batter"], subtract = True))
print()

print("Bartolo Colon + Barry Zito")
print(combine_players("Bartolo Colon", "Barry Zito", data["pitcher"], latent_vecs["pitcher"], player_names["pitcher"]))
print()

print("Bartolo Colon - Barry Zito")
print(combine_players("Bartolo Colon", "Barry Zito", data["pitcher"], latent_vecs["pitcher"], player_names["pitcher"], subtract = True))
print()

print("Barry Zito - Bartolo Colon")
print(combine_players("Barry Zito", "Bartolo Colon", data["pitcher"], latent_vecs["pitcher"], player_names["pitcher"], subtract = True))
print()

Barry Bonds + Ichiro Suzuki
[('Ichiro Suzuki', 1.0597341747849967), ('Larry Walker', 1.2526869912105965), ('Vladimir Guerrero', 1.2764425130202823), ('Luis Gonzalez', 1.2918941115773057), ('Denard Span', 1.292874997312385), ('Alex Cora', 1.3149836998412763), ('James Loney', 1.31904800724515), ('Justin Morneau', 1.3364855159838764), ('Brian Schneider', 1.3494308857061366), ('Lance Berkman', 1.3556282122921379)]

Barry Bonds - Ichiro Suzuki
[('Jim Edmonds', 1.1501339488914302), ('Rafael Palmeiro', 1.1643522540860227), ('Jason Giambi', 1.1885634392945157), ('Ken Caminiti', 1.1941923175472959), ('Jim Thome', 1.2070914893670273), ('Adam Dunn', 1.2097113021519463), ('Jorge Posada', 1.2116652692963454), ('Carlos Pena', 1.2259274790694425), ('Matt Stairs', 1.2283479082902424), ('Robin Ventura', 1.2392215137687161)]

Ichiro Suzuki - Barry Bonds
[('Emilio Bonifacio', 1.1303135934126758), ('Tony Womack', 1.176181674841251), ('Wilton Guerrero', 1.1814406481945918), ('Alex Sanchez', 1.1917717119784